# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

## Setup: Import what you'll need to scrape the page

We'll be using Selenium for this, *not* BeautifulSoup and requests.

In [1]:
import pandas as pd

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/jui/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/4r/qbtxk1s968n5kqwsz68bgc4c0000gn/T/ipykernel_15189/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [3]:
driver.get("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")

In [4]:
# options = driver.find_elements(By.TAG_NAME, "option")
# options

In [5]:
# options[9].text

In [6]:
# options[9].click()

In [7]:
Select(driver.find_element(By.ID, 'pht_status')).select_by_visible_text('Cosmetologists')

In [8]:
last_name = driver.find_element(By.ID, "pht_lnm")
last_name.send_keys("Nguyen")

In [9]:
driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div/div/section/div/div/table/tbody/tr/td/form/table/tbody/tr[18]/td/input[1]").click()

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't actual tabular data!

> You can use either Selenium by itself or Selenium+BeautifulSoup to scrape the results page. The choice is up to you!

### Loop through each result and print the entire row

Okay wait, maybe not, i's a heck of a lot of rows. Use `[:10]` to only do the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

> *Tip: If you're using Selenium, `By.TAG_NAME` is used if you don't have a class or ID. If you're using BeautifulSoup, just do your normal thing.*

In [39]:
rows = driver.find_elements(By.TAG_NAME, "tr")
# for row in rows:
#     cells = row.find_elements(By.TAG_NAME, "td")
#     print(cells[0].text)
#     print(cells[1].text)
#     print(cells[2].text)
#     print("-----") # simply did too much here
for row in rows[:10]:
    print(row.text)
    print("-----")

Name and Location Order Basis for Order
-----
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
-----
NGUYEN, DAI T
City: HOUSTON
County: Harris
Zip Code: 77034


License #: 765339

Complaint # COS20210005027 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,500. Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondent failed to store eyelash extensions in a sealed bag or covered container and kept in a clean dry d

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – `driver.find_element` or `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [49]:
for row in rows[:10]:
    try:
        print(row.find_element(By.CLASS_NAME, 'results_text').text)
    except:
        print("Doesn't have a name")

Doesn't have a name
NGUYEN, THANH
NGUYEN, DAI T
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
NGUYEN, NAM
NGUYEN, DUC
NGUYEN, THU THAO THI


## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: If you're using Selenium by itself, you can get the HTML of something by doing `.get_attribute('innerHTML')` – that way it'll look like BeautifulSoup when you print it. It might help you diagnose your issue!*
> - *Tip: Or I guess you could just skip the one with the problem...*

## Loop through each result, printing the complaint number

Output should look like this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [54]:
for row in rows[:10]:
    cell = row.find_elements(By.TAG_NAME, "td")

IndexError: list index out of range

## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> *Tip: If you want to ask for the "next sibling," you can't use `find_next_sibling` in Selenium, you need to use `element.find_element_by_xpath("following-sibling::div")` to find the next div, or `element.find_element_by_xpath("following-sibling::*")` to find the next anything.

In [82]:
rows = driver.find_elements(By.TAG_NAME, "tr")

dataset = []

for row in rows:
    data = {}
    try:
        data['name'] = row.find_element(By.CLASS_NAME, 'results_text').text
        data['city'] = row.find_element(By.CLASS_NAME, 'results_text')[1].text
    except:
        print("Doesn't have a name")
    
    dataset.append(data)
    
dataset

Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name
Doesn't have a name


[{},
 {'name': 'NGUYEN, THANH'},
 {'name': 'NGUYEN, DAI T'},
 {'name': 'NGUYEN, LONG D'},
 {'name': 'NGUYEN, LUCIE HUONG'},
 {'name': 'NGUYEN, CHINH'},
 {'name': 'NGUYEN, JIMMY'},
 {'name': 'NGUYEN, NAM'},
 {'name': 'NGUYEN, DUC'},
 {'name': 'NGUYEN, THU THAO THI'},
 {'name': 'NGUYEN, MINH NHU'},
 {'name': 'NGUYEN, DUNG VAN'},
 {'name': 'NGUYEN, TINH'},
 {'name': 'NGUYEN, HANG NU THANH'},
 {'name': 'NGUYEN, LAN N'},
 {'name': 'NGUYEN, TIEP HUY'},
 {'name': 'NGUYEN, DUNG'},
 {'name': 'NGUYEN, THAI VAN'},
 {'name': 'NGUYEN, THI THAO UYEN'},
 {'name': 'NGUYEN, THANH TRONG'},
 {'name': 'NGUYEN, KHANH V'},
 {'name': 'NGUYEN, DUC VAN'},
 {'name': 'NGUYEN, THI HOAN CHAU'},
 {'name': 'NGUYEN, PHUCHUNG'},
 {'name': 'NGUYEN, KIM'},
 {'name': 'NGUYEN, HUE VAN'},
 {'name': 'NGUYEN, HOAI THU T'},
 {'name': 'NGUYEN, NGHIA T'},
 {'name': 'NGUYEN, HANH THI'},
 {'name': 'NGUYEN, HUYEN'},
 {'name': 'NGUYEN, CHRISTINA M'},
 {'name': 'NGUYEN, THI THIBICH'},
 {'name': 'NGUYEN, KIM AN THI'},
 {'name': 'NGUY

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [67]:
import pandas as pd

pd.DataFrame(dataset)

,dict
0,Name and Location Order Basis for Order
1,"NGUYEN, THANH\nCity: FRISCO\nCounty: COLLIN\nZ..."
2,"NGUYEN, DAI T\nCity: HOUSTON\nCounty: Harris\n..."
3,"NGUYEN, LONG D\nCity: SAN SABA\nCounty: SAN SA..."
4,"NGUYEN, LUCIE HUONG\nCity: UVALDE\nCounty: UVA..."
...,...
98,"NGUYEN, SON DANG\nCity: FOREST HILL\nCounty: T..."
99,"NGUYEN, TUYEN THANH\nCity: AUBREY\nCounty: DEN..."
100,"NGUYEN, STEVEN THAI\nCity: CORSICANA\nCounty: ..."
101,"NGUYEN, CY VAN\nCity: HOUSTON\nCounty: HARRIS\..."


In [68]:
df = pd.DataFrame(dataset)
df

,dict
0,Name and Location Order Basis for Order
1,"NGUYEN, THANH\nCity: FRISCO\nCounty: COLLIN\nZ..."
2,"NGUYEN, DAI T\nCity: HOUSTON\nCounty: Harris\n..."
3,"NGUYEN, LONG D\nCity: SAN SABA\nCounty: SAN SA..."
4,"NGUYEN, LUCIE HUONG\nCity: UVALDE\nCounty: UVA..."
...,...
98,"NGUYEN, SON DANG\nCity: FOREST HILL\nCounty: T..."
99,"NGUYEN, TUYEN THANH\nCity: AUBREY\nCounty: DEN..."
100,"NGUYEN, STEVEN THAI\nCity: CORSICANA\nCounty: ..."
101,"NGUYEN, CY VAN\nCity: HOUSTON\nCounty: HARRIS\..."


In [69]:
df.to_csv("output.csv", index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [70]:
pd.read_csv("output.csv")

,dict
0,Name and Location Order Basis for Order
1,"NGUYEN, THANH\nCity: FRISCO\nCounty: COLLIN\nZ..."
2,"NGUYEN, DAI T\nCity: HOUSTON\nCounty: Harris\n..."
3,"NGUYEN, LONG D\nCity: SAN SABA\nCounty: SAN SA..."
4,"NGUYEN, LUCIE HUONG\nCity: UVALDE\nCounty: UVA..."
...,...
98,"NGUYEN, SON DANG\nCity: FOREST HILL\nCounty: T..."
99,"NGUYEN, TUYEN THANH\nCity: AUBREY\nCounty: DEN..."
100,"NGUYEN, STEVEN THAI\nCity: CORSICANA\nCounty: ..."
101,"NGUYEN, CY VAN\nCity: HOUSTON\nCounty: HARRIS\..."
